In [30]:
def xpath2xml(xpath, xmlns=''):
    
    pl = xpath.split('/')

    xmls = f'<{pl[1]} xmlns="{xmlns}">'
    xmle = f'</{pl[1]}>'
     
    def _xpath2xml(pl):
        key = ''
        xmls = ''
        xmle = ''

        for i in range(len(pl)):
            elem = pl[i]
            if "=" in elem:
                elem,key = elem.split("=")
                xmls += f'<{elem}>{key}</{elem}>'
                break
            xmls += f'<{elem}>'
            xmle = f'</{elem}>' + xmle
     
        if key and i < len(pl)-1:
            return xmls + _xpath2xml(pl[(i+1)::]) + xmle #recursion
        else:
            return xmls + xmle
     
    return xmls + _xpath2xml(pl[2::]) + xmle

In [42]:
from scrapli_netconf.driver import AsyncNetconfScrape
my_device = {
    "host": "192.168.99.91",
    "auth_username": "apiuser",
    "auth_password": "apipassword",
    "auth_strict_key": False,
    "transport": "asyncssh",
}

conn = AsyncNetconfScrape(**my_device)
await conn.open()
xmlns = "http://cisco.com/ns/yang/cisco-nx-os-device"

In [43]:
conf_tpl="""
<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bd-items>
            <bd-items>
                <BD-list>
                    <fabEncap>vlan-{vlan_id}</fabEncap>
                    <name>{vlan_name}</name>
                    <BdState>active</BdState>
                    <accEncap>vxlan-{vni}</accEncap>
                    <adminSt>active</adminSt>
                    <bridgeMode>mac</bridgeMode>
                    <fwdCtrl>mdst-flood</fwdCtrl>
                    <fwdMode>bridge,route</fwdMode>
                    <id>{vlan_id}</id>
                    <mode>CE</mode>
                </BD-list>
            </bd-items>
        </bd-items>
        <intf-items>
            <svi-items>
                <If-list>
                    <id>vlan{vlan_id}</id>
                    <mtu>{mtu}</mtu>
                    <descr>{description}</descr>
                    <adminSt>up</adminSt>
                    <rtvrfMbr-items>
                        <tDn>/System/inst-items/Inst-list[name='{vrf_name}']</tDn>
                    </rtvrfMbr-items>
                    <vlanId>{vlan_id}</vlanId>
                </If-list>
            </svi-items>
        </intf-items>
        <ipv4-items>
            <inst-items>
                <dom-items>
                    <Dom-list>
                        <name>{vrf_name}</name>
                        <if-items>
                            <If-list>
                                <id>vlan{vlan_id}</id>
                                <addr-items>
                                    <Addr-list>
                                        <addr>{ip_addr}</addr>
                                    </Addr-list>
                                </addr-items>
                            </If-list>
                        </if-items>
                    </Dom-list>
                </dom-items>
            </inst-items>
        </ipv4-items>
        <hmm-items>
            <fwdinst-items>
                <if-items>
                    <FwdIf-list>
                        <id>vlan{vlan_id}</id>
                        <adminSt>enabled</adminSt>
                        <mode>anycastGW</mode>
                    </FwdIf-list>
                </if-items>
            </fwdinst-items>
        </hmm-items>
        <eps-items>
            <epId-items>
                <Ep-list>
                    <epId>1</epId>
                    <nws-items>
                        <vni-items>
                            <Nw-list>
                                <vni>{vni}</vni>
                                <IngRepl-items>
                                    <proto>bgp</proto>
                                </IngRepl-items>
                                <suppressARP>{supARP}</suppressARP>
                            </Nw-list>
                        </vni-items>
                    </nws-items>
                </Ep-list>
            </epId-items>
        </eps-items>
        <evpn-items>
            <adminSt>enabled</adminSt>
            <bdevi-items>
                <BDEvi-list>
                    <encap>vxlan-{vni}</encap>
                    <rd>rd:unknown:0:0</rd>
                    <rttp-items>
                        <RttP-list>
                            <type>export</type>
                            <ent-items>
                                <RttEntry-list>
                                    <rtt>route-target:unknown:0:0</rtt>
                                </RttEntry-list>
                            </ent-items>
                        </RttP-list>
                        <RttP-list>
                            <type>import</type>
                            <ent-items>
                                <RttEntry-list>
                                    <rtt>route-target:unknown:0:0</rtt>
                                </RttEntry-list>
                            </ent-items>
                        </RttP-list>
                    </rttp-items>
                </BDEvi-list>
            </bdevi-items>
        </evpn-items>
    </System>
</config>
"""

In [44]:
data=dict(vlan_name="VLAN126", 
          vlan_id=126, 
          vni=10126, 
          description="svi126 by netconf", 
          vrf_name="Tenant-1", 
          ip_addr="10.1.126.254/24",
          supARP="enabled", mtu=9000)

conf = conf_tpl.format(vlan_name=data['vlan_name'], 
                                   vlan_id=data['vlan_id'], 
                                   vni=data['vni'], 
                                   description=data['description'], 
                                   vrf_name=data['vrf_name'], 
                                   ip_addr=data['ip_addr'], 
                                   supARP=data['supARP'], mtu=data.get("mtu", 1500))

response = await conn.edit_config(config=conf, target="running")
print(response.result)


<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="101">
  <ok/>
</rpc-reply>



In [41]:
path = "/System/evpn-items"
response = await conn.get(filter_=xpath2xml(path, xmlns), filter_type="subtree")
print(response.result)

<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="101">
  <data>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
      <evpn-items>
        <adminSt>enabled</adminSt>
        <bdevi-items>
          <BDEvi-list>
            <encap>vxlan-10126</encap>
            <rd>rd:unknown:0:0</rd>
            <rttp-items>
              <RttP-list>
                <type>export</type>
                <ent-items>
                  <RttEntry-list>
                    <rtt>route-target:unknown:0:0</rtt>
                  </RttEntry-list>
                </ent-items>
                <operRttAuto>route-target:as2-nn2:65000:10126</operRttAuto>
              </RttP-list>
              <RttP-list>
                <type>import</type>
                <ent-items>
                  <RttEntry-list>
                    <rtt>route-target:unknown:0:0</rtt>
                  </RttEntry-list>
                </ent-items>
                <operRttAuto>route-target:as2-nn2:65000:101

In [13]:
conn.close()

<coroutine object AsyncScrape.close at 0x7fa3d773cd40>